In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from numpy import nan

Using backend: pytorch


In [2]:
data = pd.read_csv("gnn_testTSV.tsv",header=[0],sep='\t')
data.shape

C:\Users\t-dsannyasi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(218530, 109)

In [3]:
data = data.drop(columns=['billing_contact_name_FraudRate','billing_contact_org_FraudRate',
                            'billing_contact_street_FraudRate','billing_contact_city_FraudRate',
                            'billing_contact_state_FraudRate','billing_contact_postal_FraudRate',
                            'billing_contact_country_FraudRate','billing_contact_phone_FraudRate',
                            'billing_contact_fax_FraudRate','billing_contact_email_FraudRate',
                            'registrar_status_1_FraudRate','registrar_status_2_FraudRate',
                            'registrar_status_3_FraudRate','registrar_status_4_FraudRate',
                            'registrar_status_5_FraudRate','registrar_status_6_FraudRate',])

In [4]:
data.shape

(218530, 93)

In [5]:
data.head()

,Domain,Label1,server_FraudRate,registrar_name_FraudRate,registrar_abuse_contact_phone_FraudRate,registrar_abuse_contact_email_FraudRate,registrar_iana_id_FraudRate,registrar_url_FraudRate,registrar_whois_server_FraudRate,admin_contact_name_FraudRate,...,technical_contact_fax,technical_contact_email,name_server_1,name_server_2,name_server_3,name_server_4,name_server_5,name_server_6,name_server_7,name_server_8
0,000webhostapp.co,0,0.210175,0.430079,0.291721,0.291741,0.293303,0.098213,0.120801,0.158375,...,xredacted for privacy,please query the rdds service of the registrar...,ns1.parkingcrew.net,ns2.parkingcrew.net,NaN,NaN,NaN,NaN,NaN,NaN
1,000webhostapp.com,0,0.356369,0.451270,0.356463,0.367608,0.450754,0.357980,0.446011,0.208581,...,NaN,gdpr-masking@gdpr-masked.com,dns1.000webhost.com,dns2.000webhost.com,NaN,NaN,NaN,NaN,NaN,NaN
2,000webhostapp.com,0,0.356369,0.451270,0.356463,0.367608,0.450754,0.357980,0.446011,0.208581,...,NaN,gdpr-masking@gdpr-masked.com,dns1.000webhost.com,dns2.000webhost.com,NaN,NaN,NaN,NaN,NaN,NaN
3,000webhostapp.com,1,0.356369,0.451270,0.356463,0.367608,0.450754,0.357980,0.446011,0.208581,...,NaN,gdpr-masking@gdpr-masked.com,dns1.000webhost.com,dns2.000webhost.com,NaN,NaN,NaN,NaN,NaN,NaN
4,000webhostapp.com,1,0.356369,0.451270,0.356463,0.367608,0.450754,0.357980,0.446011,0.208581,...,NaN,gdpr-masking@gdpr-masked.com,dns1.000webhost.com,dns2.000webhost.com,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cols = list(data)

In [6]:
data = data.sample(n=1000, replace=False, random_state=1)
print(data.shape)

g = data.values
t=0
for i in range(g.shape[0]):
    if g[i][1] == 1:
        t = t+1
print(t)      

t=0
for i in range(900,1000,1):
    if g[i][1] == 1:
        t = t+1
print(t)

(1000, 93)
146
16


In [7]:
domains = data['Domain']
print(domains.shape)
labels = data['Label1']
print(labels.shape)
node_features = data[['server_FraudRate', 'registrar_name_FraudRate', 'registrar_abuse_contact_phone_FraudRate', 
                      'registrar_abuse_contact_email_FraudRate', 'registrar_iana_id_FraudRate', 'registrar_url_FraudRate', 
                      'registrar_whois_server_FraudRate', 'admin_contact_name_FraudRate', 'admin_contact_org_FraudRate',
                      'admin_contact_street_FraudRate', 'admin_contact_city_FraudRate', 'admin_contact_state_FraudRate',
                      'admin_contact_postal_FraudRate', 'admin_contact_country_FraudRate', 'admin_contact_phone_FraudRate', 
                      'admin_contact_fax_FraudRate', 'admin_contact_email_FraudRate', 'registrant_contact_name_FraudRate',
                      'registrant_contact_org_FraudRate', 'registrant_contact_street_FraudRate', 
                      'registrant_contact_city_FraudRate', 'registrant_contact_state_FraudRate', 
                      'registrant_contact_postal_FraudRate', 'registrant_contact_country_FraudRate', 
                      'registrant_contact_phone_FraudRate', 'registrant_contact_fax_FraudRate', 
                      'registrant_contact_email_FraudRate', 'technical_contact_name_FraudRate', 
                      'technical_contact_org_FraudRate', 'technical_contact_street_FraudRate', 
                      'technical_contact_city_FraudRate', 'technical_contact_state_FraudRate', 
                      'technical_contact_postal_FraudRate', 'technical_contact_country_FraudRate',
                      'technical_contact_phone_FraudRate', 'technical_contact_fax_FraudRate', 
                      'technical_contact_email_FraudRate', 'name_server_1_FraudRate', 'name_server_2_FraudRate', 
                      'name_server_3_FraudRate', 'name_server_4_FraudRate', 'name_server_5_FraudRate', 
                      'name_server_6_FraudRate', 'name_server_7_FraudRate', 'name_server_8_FraudRate']] 
print(node_features.shape)
edges = data[['Domain', 'server', 'registrar_name', 'registrar_abuse_contact_phone', 
              'registrar_abuse_contact_email', 'registrar_iana_id', 'registrar_url', 'registrar_whois_server', 
              'admin_contact_name', 'admin_contact_org', 'admin_contact_street', 'admin_contact_city', 
              'admin_contact_state', 'admin_contact_postal', 'admin_contact_country', 'admin_contact_phone', 
              'admin_contact_fax', 'admin_contact_email', 'registrant_contact_name', 'registrant_contact_org', 
              'registrant_contact_street', 'registrant_contact_city', 'registrant_contact_state', 'registrant_contact_postal', 
              'registrant_contact_country', 'registrant_contact_phone', 'registrant_contact_fax', 'registrant_contact_email',
              'technical_contact_name', 'technical_contact_org', 'technical_contact_street', 'technical_contact_city', 
              'technical_contact_state', 'technical_contact_postal', 'technical_contact_country', 'technical_contact_phone', 
              'technical_contact_fax', 'technical_contact_email', 'name_server_1', 'name_server_2', 'name_server_3',
              'name_server_4', 'name_server_5', 'name_server_6', 'name_server_7', 'name_server_8' ]]
print(edges.shape)

(1000,)
(1000,)
(1000, 45)
(1000, 46)


In [8]:
edgesnp = edges.values
edges_data = np.zeros((0,47))
edges_data.shape


(0, 47)

In [9]:

for i in range(edgesnp.shape[0]):
    if i%30 == 0:
        print(i/1000*100)
    for j in range(edgesnp.shape[0]):
        
        domain1 = edgesnp[i][0]
        domain2 = edgesnp[j][0]
        
        if domain1==domain2:
            continue
            
        vector1=edgesnp[i][1:]
        vector2=edgesnp[j][1:]
        
        mask = np.equal(vector1,vector2)
        t = 0
        for k in range(mask.shape[0]):
            if mask[k] == True:
                t = t+1
        if t < 2:
            continue
            
        new = np.zeros((1,47))
        new[0][0]=i
        new[0][1]=j
        for k in range(mask.shape[0]):
            if mask[k] == True:
                new[0][k+2] = 1
        
        edges_data = np.append(edges_data, new,axis=0)
        
            
        

0.0
3.0
6.0
9.0
12.0
15.0
18.0
21.0
24.0
27.0
30.0
33.0
36.0
39.0
42.0
45.0
48.0
51.0
54.0
56.99999999999999
60.0
63.0
66.0
69.0
72.0
75.0
78.0
81.0
84.0
87.0
90.0
93.0
96.0
99.0


In [10]:
edges_data = edges_data.astype('int32')
srcDst = pd.DataFrame(edges_data[:,:2],columns=['Src','Dst'])
print(srcDst.shape, edges_data.shape)
srcDst.head()


(94010, 2) (94010, 47)


,Src,Dst
0,0,3
1,0,29
2,0,32
3,0,71
4,0,247


In [11]:
edges_features = pd.DataFrame(edges_data[:,2:],columns = ['server', 'registrar_name', 'registrar_abuse_contact_phone', 
              'registrar_abuse_contact_email', 'registrar_iana_id', 'registrar_url', 'registrar_whois_server', 
              'admin_contact_name', 'admin_contact_org', 'admin_contact_street', 'admin_contact_city', 
              'admin_contact_state', 'admin_contact_postal', 'admin_contact_country', 'admin_contact_phone', 
              'admin_contact_fax', 'admin_contact_email', 'registrant_contact_name', 'registrant_contact_org', 
              'registrant_contact_street', 'registrant_contact_city', 'registrant_contact_state', 'registrant_contact_postal', 
              'registrant_contact_country', 'registrant_contact_phone', 'registrant_contact_fax', 'registrant_contact_email',
              'technical_contact_name', 'technical_contact_org', 'technical_contact_street', 'technical_contact_city', 
              'technical_contact_state', 'technical_contact_postal', 'technical_contact_country', 'technical_contact_phone', 
              'technical_contact_fax', 'technical_contact_email', 'name_server_1', 'name_server_2', 'name_server_3',
              'name_server_4', 'name_server_5', 'name_server_6', 'name_server_7', 'name_server_8'])

print(edges_features.shape)
edges_features.head()

(94010, 45)


,server,registrar_name,registrar_abuse_contact_phone,registrar_abuse_contact_email,registrar_iana_id,registrar_url,registrar_whois_server,admin_contact_name,admin_contact_org,admin_contact_street,...,technical_contact_fax,technical_contact_email,name_server_1,name_server_2,name_server_3,name_server_4,name_server_5,name_server_6,name_server_7,name_server_8
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# domains.to_csv('domainsGNN.csv')
# labels.to_csv('labelsGNN.csv')
# node_features.to_csv('nfGNN.csv')
# edges_features.to_csv('efGNN.csv')
# srcDst.to_csv('srcdstGNN.csv')

In [12]:
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
import os
from tqdm import tqdm

print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")

Torch version: 1.9.0
Cuda available: False
Torch geometric version: 1.7.2


In [13]:
Y = torch.tensor(labels.values, dtype=torch.int16)
nodef = torch.tensor(node_features.values, dtype=torch.float)
edge = torch.tensor(srcDst.values, dtype=torch.long).t().view(2,-1)
edgef = torch.tensor(edges_features.values, dtype=torch.float)
print(Y.shape, nodef.shape, edge.shape,edgef.shape)

torch.Size([1000]) torch.Size([1000, 45]) torch.Size([2, 94010]) torch.Size([94010, 45])


In [14]:
graph = Data(x=nodef, edge_index=edge, edge_attr=edgef, y=Y)

In [15]:
print(graph)

Data(edge_attr=[94010, 45], edge_index=[2, 94010], x=[1000, 45], y=[1000])


In [16]:
n_nodes = node_features.shape[0]
n_train = int(n_nodes * 0.9)
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
train_mask[:n_train] = True
test_mask[n_train:] = True

In [17]:
from torch_geometric.nn import CGConv, TransformerConv

In [18]:
class Fraud(torch.nn.Module):
    def __init__(self):
        super(Fraud, self).__init__()
        
        self.conv1 = TransformerConv(in_channels = 45, out_channels=100, heads=3, concat=False, beta=True, edge_dim=45)
        self.conv2 = TransformerConv(in_channels = 100, out_channels=100, heads=3, concat=False,beta=True, edge_dim=45)
        self.conv3 = TransformerConv(in_channels = 100, out_channels=50, heads=3, concat=False,beta=True, edge_dim=45)
        
        self.post_mp = nn.Sequential(
            nn.Linear(50,100), nn.ReLU(), nn.Dropout(0.25), 
            nn.Linear(100, 2))

    def forward(self):
        #x, edge_index, edge_attr = graph.x, graph.edge_index, graph.edge_attr
        
        x = self.conv1(x=graph.x, edge_index=graph.edge_index, edge_attr=graph.edge_attr)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.conv2(x=x, edge_index=graph.edge_index, edge_attr=graph.edge_attr)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.conv3(x=x, edge_index=graph.edge_index, edge_attr=graph.edge_attr)
        x = self.post_mp(x)
        return x
    


In [19]:
def evaluate(model, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model()
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
import sklearn
from sklearn.metrics import roc_auc_score, average_precision_score, plot_precision_recall_curve
import matplotlib.pyplot as plt
%matplotlib inline

a = range(1,11,1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model, graph = Fraud().to(device), graph.to(device)

model = Fraud()

opt = torch.optim.Adam(model.parameters())

n = 100
X = np.arange(0,n,1)
roc = np.zeros(n)
pr = np.zeros(n)
accu = np.zeros(n)

for epoch in range(n):
    
    model.train()
    # forward propagation by using all nodes
    logits = model()
    # compute loss
    loss = F.cross_entropy(logits[train_mask].float(), Y[train_mask].long(), weight=torch.tensor([1,5]).float())

    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    model.eval()
    acc = evaluate(model, Y, test_mask)
    
    if epoch%10 == 0:
        print('Epoch:',epoch,' Loss:',loss.item())


    logits = model()
    logits = F.softmax(logits,dim=1)
    logits = logits[test_mask]
    labels = Y[test_mask]
    numpy_logits = logits.detach().numpy()
    
    roc[epoch] = roc_auc_score(labels, numpy_logits[:,1])
    pr[epoch] = average_precision_score(labels, numpy_logits[:,1])
    accu[epoch] = acc
    
plt.plot(X,roc)
plt.show()
plt.plot(X,pr)
plt.show()
plt.plot(X,accu)
plt.show()

Epoch: 0  Loss: 0.6896312236785889
Epoch: 10  Loss: 0.6640636920928955
Epoch: 20  Loss: 0.6145026087760925
Epoch: 30  Loss: 0.5810387134552002
Epoch: 40  Loss: 0.5546746253967285
Epoch: 50  Loss: 0.5396450161933899
Epoch: 60  Loss: 0.5226187109947205
Epoch: 70  Loss: 0.5089074373245239
Epoch: 80  Loss: 0.4928551912307739


In [ ]:
rocpd = pd.DataFrame(roc,columns=['val'])
prpd = pd.DataFrame(pr,columns=['val'])
accupd = pd.DataFrame(accu,columns=['val'])

In [ ]:
rocavg = rocpd.val.rolling(5).mean()
pravg = prpd.val.rolling(5).mean()
accuavg = accupd.val.rolling(5).mean()

In [ ]:
plt.plot(X,roc)
plt.plot(X,rocavg)
plt.ylabel('roc')
plt.show()
plt.plot(X,pr)
plt.plot(X,pravg)
plt.ylabel('pr')
plt.show()
plt.plot(X,accu)
plt.plot(X,accuavg)
plt.ylabel('acc')
plt.show()

In [132]:
# Specify a path
PATH = "state_dict_model.pt"

# Save
torch.save(model.state_dict(), PATH)

In [ ]:
# Load
model = Fraud()
model.load_state_dict(torch.load(PATH))
model.eval()